In [10]:
from math import sin, cos

from gaft import GAEngine
from gaft.components import BinaryIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation

# Analysis plugin base class.
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis

# Built-in best fitness analysis.
from gaft.analysis.fitness_store import FitnessStore

# Define population.
indv_template = BinaryIndividual(ranges=[(0, 10)], eps=0.001)
population = Population(indv_template=indv_template, size=30).init()

# Create genetic operators.
selection = TournamentSelection()
crossover = UniformCrossover(pc=0.8, pe=0.5)
mutation = FlipBitMutation(pm=0.1)

# Create genetic algorithm engine.
engine = GAEngine(population=population, selection=selection,
                  crossover=crossover, mutation=mutation,
                  analysis=[FitnessStore])

# Define fitness function.
@engine.fitness_register
def fitness(indv):
    x, = indv.solution
    return x - 8*sin(5*x) + 7*cos(7*x)

# Define on-the-fly analysis.
@engine.analysis_register
class ConsoleOutputAnalysis(OnTheFlyAnalysis):
    interval = 1
    master_only = True

    def register_step(self, g, population, engine):
        best_indv = population.best_indv(engine.fitness)
        msg = 'Generation: {}, best fitness: {:.3f}'.format(g, engine.ori_fmax)
        self.logger.info(msg)

    def finalize(self, population, engine):
        best_indv = population.best_indv(engine.fitness)
        x = best_indv.solution
        y = engine.ori_fmax
        msg = 'Optimal solution: ({}, {})'.format(x, y)
        self.logger.info(msg)

if '__main__' == __name__:
    # Run the GA engine.
    engine.run(ng=100)

gaft.ConsoleOutputAnalysis   INFO     Generation: 0, best fitness: 23.224
gaft.ConsoleOutputAnalysis   INFO     Generation: 1, best fitness: 23.325
gaft.ConsoleOutputAnalysis   INFO     Generation: 2, best fitness: 23.325
gaft.ConsoleOutputAnalysis   INFO     Generation: 3, best fitness: 23.595
gaft.ConsoleOutputAnalysis   INFO     Generation: 4, best fitness: 23.607
gaft.ConsoleOutputAnalysis   INFO     Generation: 5, best fitness: 23.607
gaft.ConsoleOutputAnalysis   INFO     Generation: 6, best fitness: 23.618
gaft.ConsoleOutputAnalysis   INFO     Generation: 7, best fitness: 23.618
gaft.ConsoleOutputAnalysis   INFO     Generation: 8, best fitness: 23.618
gaft.ConsoleOutputAnalysis   INFO     Generation: 9, best fitness: 23.618
gaft.ConsoleOutputAnalysis   INFO     Generation: 10, best fitness: 23.618
gaft.ConsoleOutputAnalysis   INFO     Generation: 11, best fitness: 23.618
gaft.ConsoleOutputAnalysis   INFO     Generation: 12, best fitness: 23.618
gaft.ConsoleOutputAnalysis   INFO  

In [14]:
import random
def fuzzer(max_length=100, char_start=32, char_range=32):
    """A string of up to `max_length` characters
       in the range [`char_start`, `char_start` + `char_range`]"""
    string_length = random.randrange(0, max_length + 1)
    out = ""
    for i in range(0, string_length):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

In [17]:
fuzzer()

"(-.$?7*-,'==/):6 '/$"

In [18]:
fuzzer(1000, ord('a'), 26)

'orxjksmrtejifuiyvwmxtpbmlbfbpxvjryeqbyafzisfoejzordtyintgsyrewlthpiszknrjbjxhlxqmmmowskoinzewtglxsslenmiaoojmpwqixvxngnazgnjgevreahuikwvmmszqcnnc'

In [23]:
import os
import tempfile
basename = "input.txt"
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
print(FILE)

data = fuzzer()
with open(FILE, "w") as f:
    f.write(data)
    
contents = open(FILE).read()
print(contents)
assert(contents == data)

C:\Users\ETSUKE~1\AppData\Local\Temp\tmponc2ls_e\input.txt
%/>.<<('>2%'*:!#.).%9$=)!?(15,2<!=5*.?/


In [24]:
import os
import subprocess
program = "bc"
with open(FILE, "w") as f:
    f.write("2 + 2\n")
result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)  # Will be "text" in Python 3.7

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [25]:
from urllib.parse import urlparse
urlparse("http://www.google.com/search?q=fuzzing")

ParseResult(scheme='http', netloc='www.google.com', path='/search', params='', query='q=fuzzing', fragment='')

In [29]:
import sys
class Coverage(object):
    # Trace function
    def traceit(self, frame, event, arg):
        if self.original_trace_function is not None:
            self.original_trace_function(frame, event, arg)

        if event == "line":
            function_name = frame.f_code.co_name
            lineno = frame.f_lineno
            self._trace.append((function_name, lineno))

        return self.traceit

    def __init__(self):
        self._trace = []

    # Start of `with` block
    def __enter__(self):
        self.original_trace_function = sys.gettrace()
        sys.settrace(self.traceit)
        return self

    # End of `with` block
    def __exit__(self, exc_type, exc_value, tb):
        sys.settrace(self.original_trace_function)

    def trace(self):
        """The list of executed lines, as (function_name, line_number) pairs"""
        return self._trace

    def coverage(self):
        """The set of executed lines, as (function_name, line_number) pairs"""
        return set(self.trace())

In [30]:
def cgi_decode(s):
    """Decode the CGI-encoded string `s`:
       * replace "+" by " "
       * replace "%xx" by the character with hex number xx.
       Return the decoded string.  Raise `ValueError` for invalid inputs."""

    # Mapping of hex digits to their integer values
    hex_values = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15,
        'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
    }

    t = ""
    i = 0
    while i < len(s):
        c = s[i]
        if c == '+':
            t += ' '
        elif c == '%':
            digit_high, digit_low = s[i + 1], s[i + 2]
            i += 2
            if digit_high in hex_values and digit_low in hex_values:
                v = hex_values[digit_high] * 16 + hex_values[digit_low]
                t += chr(v)
            else:
                raise ValueError("Invalid encoding")
        else:
            t += c
        i += 1
    return t

In [32]:
with Coverage() as cov:
    print("1")

print(cov.coverage())

1
{('_schedule_flush', 332), ('is_alive', 1112), ('_wait_for_tstate_lock', 1069), ('send', 390), ('_schedule_flush', 328), ('write', 402), ('_wait_for_tstate_lock', 1070), ('write', 410), ('_event_pipe', 97), ('_schedule_flush', 329), ('is_alive', 1115), ('_wait_for_tstate_lock', 1072), ('schedule', 205), ('write', 403), ('send', 395), ('write', 385), ('is_set', 508), ('_is_master_process', 310), ('_event_pipe', 105), ('_event_pipe', 96), ('_schedule_flush', 334), ('write', 400), ('_schedule_flush', 327), ('write', 393), ('__exit__', 26), ('schedule', 203), ('write', 397), ('is_alive', 1113), ('send', 400), ('is_alive', 1116), ('schedule', 202)}
